# Test the Model & Visualize it 
Now the model is craeted an trained. So let's use it testwise. Idea would be to 
- be able to run it - since it works on the daily candle - on a daily base
- store the outcome of the test in a table to be able to 
- visualize it

In [1]:
# here we convert the jupiter file to a python script since it is easier to handle 
!jupyter nbconvert --to script --output test_the_model test_the_model.ipynb

[NbConvertApp] Converting notebook test_the_model.ipynb to script
[NbConvertApp] Writing 7129 bytes to test_the_model.py


Let's get the libs we need

In [2]:
import pandas as pd
import os
import joblib

from stable_baselines3 import PPO
from supporting_classes import TradingEnv, OHLCScaler

[NbConvertApp] Converting notebook supporting_classes.ipynb to script
[NbConvertApp] Writing 6532 bytes to supporting_classes.py


We also need the data to play with

In [3]:
# at first, get a bit of data to play with
# Load raw data
data = pd.read_csv('../data/Candlestick_01nov1999_28oct2025.csv', parse_dates=['date'], index_col='date')

# Split into train/test again, now we only need the test data
split_idx  = int(len(data) * 0.8)
test_data  = data.iloc[split_idx:]

Now create some Global Varibles

In [4]:
WINDOW_SIZE = 50

# create the moste probable model name (model was stored with the last date of the training data)
filePath = f"../models/ppo_trading_{data.iloc[:split_idx].index[split_idx - 1].strftime('%Y_%m_%d')}.zip"

# check if the model exists
if os.path.exists(filePath):
    model = PPO.load(filePath)              # load the trained model
    print(f"Loading model from {filePath}") # state what we you did
else:       
    print(f"ERROR: Path {filePath} doesn't exist")
    print("ERROR:    ")
    print("ERROR:    Will terminate this execution!")
    exit()

# load the scaler for (re-)normalizing the data
scaler = joblib.load("../models/scaler.pkl")

# get some "gloabl" variables ready for later analysis and visualisation
BET            = 1000.0  # amount to invest per trade
todays_date    = test_data.index[0]
todays_predict = 0 # (0 = HOLD, 1 = BUY, 2 = SELL)
bank_account   = 0.0

log_df = pd.DataFrame({
    "Date": pd.Series(dtype="str"),
    "Predicted_Action": pd.Series(dtype="str"),
    "Real_Autcome": pd.Series(dtype="str"),
    "Stock_Profit_Loss": pd.Series(dtype="float"),
    "Bank_Account": pd.Series(dtype="float")
})


Loading model from ../models/ppo_trading_2020_08_13.zip


### Functions
Here I'd like to declare some functions to make things mor visible

This function will return the OHLC-Volume data for the given date (+ the ammount of WINDOW_SIZE previous data points)   

In [5]:

# Grab into the test data and return the OHLC+Valume for the given (assuming todays) date
# INPUT: date_str - date as string in format 'YYYY-MM-DD'
# OUTPUT: panda series with OHLC+Valume for the given date in the form of
#           date          Open        High        Low         Close        Volume
#           2020-08-07    2.357494    2.386419    2.343762    2.382781    -0.739343
# OUTPUT: next_date - the next possible date fro trading
def get_todays_outcome(date_str, window_size=WINDOW_SIZE):
    # If the given date is not in the index, find the next available date
    if date_str not in data.index:
        print(f"Couldn't find date: {date_str}")
        # Find the next date in the index that is greater than the given date
        later_dates = data.index[data.index > date_str]
        if later_dates.empty:
            print("No later date available.")
            return None
        next_date = later_dates[0]
        print(f"Using next available date: {next_date}")
    else:
        # Get the index of the given date
        idx = data.index.get_loc(date_str)
        # Check if there is a next date available
        if idx + 1 >= len(data.index):
            print("No next date available.")
            return None
        next_date = data.index[idx + 1]

    # Get the index of the actual (available) date
    idx = data.index.get_loc(next_date)

    # Check if there is enough data before the selected date
    if idx < (window_size + 1):
        print(f"Not enough data before {next_date} for window_size={window_size}.")
        return None

    print(f"Returning data for: {next_date}")
    # Return the sequence and the next available date. we need one more date the window size!!
    return data.iloc[idx - window_size - 1:idx], next_date

This function will do **ONE** step through the day

In [6]:
# This function does one step of action-prediction and returns the reward and new prediction
# INPUT: todays_predict - what was predicted for today (0 = HOLD, 1 = BUY, 2 = SELL)
# INPUT: todays_outcome - the actual OHLC+Volume data for today
# OUTPUT: new_reward - the reward received for the action taken
# OUTPUT: new_prediction - the new prediction for tomorrow (0 = HOLD, 1 = BUY, 2 = SELL)
def do_one_step_and_make_prediction(todays_predict, todays_outcome, window_size):
    
    scaled_df = scaler.transform(todays_outcome)                    # scale the data

    temp_env = TradingEnv(scaled_df, window_size=window_size)   # create a temporary environment for the given data
    obs, _ = temp_env.reset()                                   # reset the environment to get the initial observation

    new_prediction, _ = model.predict(obs)                              # use the model to predict the action based on the observation
    obs, reward, done, _, _ = temp_env.step(todays_predict)
    
    return reward, new_prediction

This function will calculate the money gained or lost based on the given "reward"

In [7]:
def calculate_stock_profit_loss(reward):
    profit = reward * BET
    return profit

This fct will visualize the given data

In [8]:

import matplotlib.pyplot as plt

def visualize_results(log_df):
    fig, ax1 = plt.subplots(figsize=(12, 6))

    # Linke Y-Achse: Bankkonto
    ax1.plot(log_df['date'], log_df['bank_account'], color='blue', label='Bankkonto')
    ax1.set_xlabel('Datum')
    ax1.set_ylabel('Bankkonto', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')

    # Rechte Y-Achse: Börsenkurs
    ax2 = ax1.twinx()
    ax2.plot(log_df['date'], log_df['real_outcome'], color='green', label='Börsenkurs')
    ax2.set_ylabel('Börsenkurs', color='green')
    ax2.tick_params(axis='y', labelcolor='green')

    # Titel und Layout
    plt.title('Bankkonto vs. Börsenkurs')
    fig.tight_layout()
    plt.show()


# The Main Test Function

Whenever this block is called, it'll loop one day through the model and log + visualize the outcome

In [9]:
# Let's do one day. Lets get the values we found for todays date
todays_outcome, next_days_date = get_todays_outcome(todays_date, WINDOW_SIZE)

todays_reward, tomorrows_predict = do_one_step_and_make_prediction(todays_predict, todays_outcome, WINDOW_SIZE)

profit_loss   = calculate_stock_profit_loss(todays_reward)
bank_account += profit_loss

# log it all
new_row = {
    "Date": todays_date,
    "Predicted_Action": todays_predict,
    "Real_Autcome": todays_outcome['Close'].iloc[-1],  
    "Stock_Profit_Loss": profit_loss,
    "Bank_Account": bank_account
}

log_df = pd.concat([log_df, pd.DataFrame([new_row])], ignore_index=True)

print(log_df.tail())

todays_date    = next_days_date
todays_predict = tomorrows_predict


Returning data for: 2020-08-17 00:00:00


AttributeError: 'TradingEnv' object has no attribute '_calculate_reward'